<a href="https://colab.research.google.com/github/gorogoro-uk/TensorFlow/blob/master/TensorFlow_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Transer Learning**

In [ ]:
# Pretrained Inception Model for Image Classification

# download model weight file from Google, save as H5 file
INCEP_URL = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
INCEP_WGTS = BASE / 'incep_v3_weights.h5'

# instantiate a model object
inception_model = InceptionV3(input_shape = (150, 150, 3),
                                include_top = False,
                                weights = None)

# load pre-trianed weights into model object
inception_model.load_weights(INCEP_WGTS)

# make layers non-trainable
for layer in inception_model.layers:
  layer.trainable = False

# define last layer of pre-trained model to use
last_layer = inception_model.get_layer('mixed7')
last_output = last_layer.output

# define customised model head to attached to pre-trained base
# weights will be trainable 
x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# join two models together
# ready for training
horsehuman_model = tf.keras.Model(inception_model.input, x)

In [ ]:
# Pretrained Word Embeddings for NLP

# download pretrained word emeddings as a file from internet
GLOVE_FILE = BASE / 'glove.6B.100d.txt'
GLOVE_URL = 'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt'
urllib.request.urlretrieve(GLOVE_URL, GLOVE_FILE)

# read embeddings from file into embed_index dict {word: embedding_vector}
# has embeddings for a huge volume of words
embed_index = {}
with open(GLOVE_FILE) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embed_index[word] = coefs

# prepare embedding matrix of weights for model shape [vocab_size, embedding_dimension]
# only get embeddings for words in the word_list for the given task
embed_matrix = np.zeros((vocab_size, embed_dim))      
for word, i in word_index.items():
    embed_vector = embed_index.get(word)
    if embed_vector is not None:
        embed_matrix[i] = embed_vector

# load weights into Embedding layer, make non-trainable
# input: [batch, sentence_length]
glove_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1, 
                              embed_dim, 
                              input_length=max_length, 
                              weights=[embed_matrix], 
                              trainable=False),
                              # output: [batch, sentence_length, embedding_dimension]
])